<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Quantitative Finance

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

# `pandas` package

In [ ]:
import sys
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'svg'

## Financial Data

In [ ]:
url = 'https://certificate.tpq.io/mlfin.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
raw.info()

In [ ]:
print(raw.head())

In [ ]:
print(raw.tail())

In [ ]:
print(raw.describe())

## Visualization

In [ ]:
raw['BTC='].dropna().plot();

In [ ]:
raw['BTC='].dropna().iloc[1000:].plot();

In [ ]:
raw['BTC='].dropna().loc['2017-7-1':].plot();

In [ ]:
raw[['AAPL.O', 'AMZN.O']].dropna().plot();

In [ ]:
(raw[['AAPL.O', 'AMZN.O']] / raw[['AAPL.O', 'AMZN.O']].iloc[0]).dropna().plot();

In [ ]:
raw[['.SPX', '.VIX']].dropna().plot(subplots=True);

In [ ]:
raw[['.SPX', '.VIX']].dropna().pct_change().hist(bins=50);

In [ ]:
raw[['.SPX', '.VIX']].dropna().pct_change().corr()

In [ ]:
raw[['.SPX', '.VIX']].dropna().plot(secondary_y='.VIX', lw=1, alpha=0.8);

## Adding Statistics

In [ ]:
data = pd.DataFrame(raw['EUR=']).dropna()

In [ ]:
data.head()

In [ ]:
data['min'] = data['EUR='].rolling(42).min()

In [ ]:
data['sma'] = data['EUR='].rolling(42).mean()

In [ ]:
data['max'] = data['EUR='].rolling(42).max()

In [ ]:
data.info()

In [ ]:
data.iloc[-750:].plot(lw=1, style=['-', 'g--', 'r-.', 'g--']);

## Interactive Visualization

**<p style="color:red;">The following does not work as it is on Google Colab.</p>**

In [ ]:
import cufflinks
cufflinks.set_config_file(offline=True)

In [ ]:
data.iloc[-1500:].iplot();

In [ ]:
ohlc = raw['AAPL.O'].resample('1w', label='right').ohlc()

In [ ]:
ohlc.head()

In [ ]:
qf = cufflinks.QuantFig(ohlc.iloc[-100:], title='Apple Stock',
                        legend='top', name='AAPL.O',
                        up_color='green', down_color='red')

In [ ]:
qf.iplot()

In [ ]:
raw.iloc[:, :4].pct_change().iplot(kind='hist', subplots=True, bins=50);

In [ ]:
raw.pct_change().corr().iplot(kind='heatmap', colorscale='Reds');

## Streaming Visualization

In [ ]:
import zmq
import time
import random
import datetime as dt
from multiprocess import Process
import plotly.graph_objects as go

In [ ]:
def tick_serv():
    context = zmq.Context()
    socket = context.socket(zmq.PUB)
    socket.bind('tcp://127.0.0.1:5555')
    PRICE = 100.
    while True:
        PRICE += random.gauss(0, 1) / 2
        msg = {'PRICE': PRICE}
        socket.send_pyobj(msg)
        time.sleep(random.random() * 2)

In [ ]:
process = Process(target=tick_serv)

In [ ]:
process.start()

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://127.0.0.1:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [ ]:
fig = go.FigureWidget()
fig.add_scatter()
fig

In [ ]:
times = list()
prices = list()
while True:
    msg = socket.recv_pyobj()
    t = dt.datetime.now()
    times.append(t)
    price = msg['PRICE']
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="mailto:training@tpq.io">training@tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> 